# Publicly-available Cebuano Corpus (An Exploration)

In [1]:
import json
import os
import sys
import pandas as pd
import matplotlib

## Wikipedia Corpus

In [2]:
text = []
for line in open('../text/AA/wiki_00', 'r'):
    text.append(json.loads(line))   

print first few lines

In [3]:
text[:2]

[{'id': '965',
  'url': 'https://ceb.wikipedia.org/wiki?curid=965',
  'title': 'Sugbo',
  'text': 'Sugbo\n\nAng Sugbo (Kinatsila: "Cebú") usa ka lalawigan sa Tunga-tungang Kabisay-an sa Pilipinas. Ang kapital ug kinadak-ang dakbayan sa lalawigan mao ang Dakbayan sa Sugbo. Ang lalawigan gitibuok sa mga pulo sa Sugbo, Mactan, Malapascua, Bantayan, Olango, ug Camotes.\n\nNahimutang kini sa tunga-tungang bahin sa nasod - sidlakan sa pulo sa Negros, sa kasadpan sa pulo sa Leyte ug amihanang-kasadpan sa pulo sa Bohol (9°25\'N-11°15\'N latitud ug 123°13\'E-124°5\'E longitud).\n\nAng kapital nga Dakbayan sa Sugbo, ang nahiunang dakbayan sa Pilipinas, bahin sa Metropolitan nga Sugbo apil ang unom nga tapad nga mga dakbayan ug unom ka mga lungsod. Ang tugpahanan sa lalawigan mao ang Mactan-Cebu International Airport sa pulo sa Mactan, ang ikaduhang labing importante nga tugpahanan sa nasod. 85% sa mga "shipping line" sa Pilipinas nagbase sa Sugbo. Ang lalawigan may unom ka mga pantalan nga nasyo

In [11]:
df = pd.DataFrame(text)
df['length'] = df['text'].apply(len)

In [14]:
print('Total current article count is', df.shape[0])
print('Total character length is', df.length.sum())

Total current article count is 594
Total character length is 1961877


In [13]:
df['length'].describe()

count      594.000000
mean      3302.823232
std       4451.175106
min       1208.000000
25%       1560.500000
50%       2205.500000
75%       3191.250000
max      60068.000000
Name: length, dtype: float64

## Oscar Cebuano Corpus

In [66]:
lines_oscar = []
with open('../shuff-dedup/ceb/ceb_dedup.txt.gz', 'rb') as f:
    for line in f:
        lines_oscar.append(line.decode('utf-8'))


In [7]:
f'The OSCAR corpus contains about {len(lines_oscar)} lines of text.' 

'The OSCAR corpus contains about 75220 lines of text.'

In [67]:
lines_oscar[:10]

['matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado\n',
 'Maayong kaagahon liwat diri sa payag ICCC...nakaluy-an sa guihapon ang aton entry nga maka puntos sa ikaduha nga sulatada....goodluck sa madason pa guid nga mga sulatadas.\n',
 'tani ari si pre idol toto jericp SR, para siya ang mangkoton ta. hehehe. bal an ya na kung sin o ang champion. hehehe\n',
 'Kapital sa munisipyo ang Āmol (Pinulongang Persiyano: آمل) sa Iran.[1] Nahimutang ni sa lalawigan sa Ostān-e Māzandarān, sa amihanang bahin sa nasod, 120 km sa amihanan-sidlakan sa Tehrān ang ulohan sa nasod. 96 metros ibabaw sa dagat kahaboga ang nahimutangan sa Āmol[1], ug adunay 199,382 ka molupyo.[1]\n',
 '↑ Kalkulado gikan sa pakigbingkil sa tanan nga gitas-on data (DEM 3") gikan sa Viewfinder Panoramas, sa sulod sa 10 ka kilometro radius.[2] Ang bug-os nga algoritmo anaa dinhi.\n',
 'Ang yuta palibot sa Āmol kay kasagaran patag, apan sa habagatan nga kini mao ang kab

In [42]:
from tokenizers import normalizers, TextInputSequence, Tokenizer
from tokenizers.normalizers import NFD, NFKC, StripAccents, Lowercase, Sequence
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

### Test normalization

In [29]:
normalizer = normalizers.Sequence([NFD(), StripAccents(), Lowercase()])
for line in lines_oscar[:5]:
    print(normalizer.normalize_str(line.decode('utf-8')))

matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado

maayong kaagahon liwat diri sa payag iccc...nakaluy-an sa guihapon ang aton entry nga maka puntos sa ikaduha nga sulatada....goodluck sa madason pa guid nga mga sulatadas.

tani ari si pre idol toto jericp sr, para siya ang mangkoton ta. hehehe. bal an ya na kung sin o ang champion. hehehe

kapital sa munisipyo ang amol (pinulongang persiyano: امل) sa iran.[1] nahimutang ni sa lalawigan sa ostan-e mazandaran, sa amihanang bahin sa nasod, 120 km sa amihanan-sidlakan sa tehran ang ulohan sa nasod. 96 metros ibabaw sa dagat kahaboga ang nahimutangan sa amol[1], ug adunay 199,382 ka molupyo.[1]

↑ kalkulado gikan sa pakigbingkil sa tanan nga gitas-on data (dem 3") gikan sa viewfinder panoramas, sa sulod sa 10 ka kilometro radius.[2] ang bug-os nga algoritmo anaa dinhi.



### Create a pipeline

In [73]:
tokenize = Tokenizer(BPE())
tokenize.normalizer = Sequence([
    NFD(),
    StripAccents(),
    Lowercase()
])
tokenize.pre_tokenizer = ByteLevel()
tokenize.decoder = ByteLevelDecoder()

In [74]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenize.train(["../shuff-dedup/ceb/ceb_dedup.txt.gz"], trainer)

Take it for a spin, use it on the first three lines

In [76]:
tokenize.encode('wassap').ids

[1172, 13263]

In [82]:
for line in lines_oscar[:3]:
    encoded = tokenize.encode(line)
    print('token ids', encoded.ids)
    print('original line:', line)
    print('decoded:', tokenize.decode(encoded.ids))


token ids [11295, 319, 335, 70, 604, 555, 352, 419, 1313, 259, 631, 2391, 1772, 967, 11668, 291, 10979, 82, 11, 500, 8079, 2172, 410, 2299, 433, 291, 2936, 17611, 67, 335, 70, 604, 955, 3395, 545, 198]
original line: matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado

decoded:  matamwa lg ko mayung gabe sa tanan....balikbayan derby na megs, 5cock daw 3 & 4 na bla......d lg ko kasigurado

token ids [1524, 2403, 22803, 6399, 3075, 259, 10312, 14266, 1130, 13544, 394, 88, 12, 256, 259, 16282, 1471, 272, 2743, 8416, 270, 458, 10354, 259, 3638, 270, 7168, 279, 306, 2391, 21522, 11592, 259, 282, 20652, 302, 10735, 270, 285, 7168, 279, 395, 13, 198]
original line: Maayong kaagahon liwat diri sa payag ICCC...nakaluy-an sa guihapon ang aton entry nga maka puntos sa ikaduha nga sulatada....goodluck sa madason pa guid nga mga sulatadas.

decoded:  maayong kaagahon liwat diri sa payag iccc...nakaluy-an sa guihapon ang aton entry nga maka 